In [ ]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
! nvidia-smi

Sun Mar 29 13:52:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Import; installation

In [ ]:
import os
import sys
!pip install gdown
import gdown
from datetime import datetime

from pathlib import Path

In [ ]:
mmdetection_repo = 'https://github.com/open-mmlab/mmdetection'
if not os.path.exists('mmdetection/'):
    !git clone -q {mmdetection_repo}
    # gdown.download(url_pikachu_dataset, output_pikachu_dataset, False)
    # gdown.download(url_init, output_init, False)
    # gdown.download(url_config, output_config, False)
    !cd mmdetection/ && pip install -q -r requirements.txt
    !cd mmdetection/ && python setup.py develop 
    sys.path.append(os.path.abspath('mmdetection/'))

Streaming output truncated to the last 5000 lines.
   T * data() const {
 ^ ~~
mmdet/ops/sigmoid_focal_loss/src/sigmoid_focal_loss_cuda.cu:160:423: warning: ‘T* at::Tensor::data() const [with T = long int]’ is deprecated [-Wdeprecated-declarations]
   AT_DISPATCH_FLOATING_TYPES_AND_HALF(
                                                                                                                                                                                                                                                                                                                                                                                                                                       ^
/usr/local/lib/python3.6/dist-packages/torch/include/ATen/core/TensorBody.h:322:1: note: declared here
   T * data() const {
 ^ ~~
mmdet/ops/sigmoid_focal_loss/src/sigmoid_focal_loss_cuda.cu:160:467: warning: ‘T* at::Tensor::data() const [with T = double]’ is deprecated [-Wdeprecated-d

In [ ]:
!pip install --upgrade Pillow

     |████████████████████████████████| 2.1MB 9.3MB/s eta 0:00:01
ERROR: mmdet 1.1.0+883dedc has requirement Pillow<=6.2.2, but you'll have pillow 7.0.0 which is incompatible.
  Found existing installation: Pillow 6.2.2
    Uninstalling Pillow-6.2.2:
      Successfully uninstalled Pillow-6.2.2


In [ ]:
from __future__ import division
import argparse
import os

import torch
from mmcv import Config

from mmdetection.mmdet import __version__
from mmdet.apis import (get_root_logger, set_random_seed, # TODO: init_dist
                        train_detector)
from mmdet.datasets import build_dataset
from mmdet.models import build_detector

# Data loading

In [ ]:
# google drive version

project_main_dir = Path('./drive/My Drive/Colab Notebooks/Studia (DNN)/projekt/')
data_main = project_main_dir / 'data'
output_data = data_main / 'stanford'
train_location = output_data / 'cars_train'
labels_url = 'https://raw.githubusercontent.com/morganmcg1/stanford-cars/master/labels_df.csv'

In [ ]:
# # downloading version:

# url_data = 'http://imagenet.stanford.edu/internal/car196/cars_train.tgz'
# train_file = 'data/stanford/cars_train.tgz'
# data_main = Path('./data/')
# output_data = data_main / 'stanford'
# train_location = output_data / 'cars_train'
# labels_url = 'https://raw.githubusercontent.com/morganmcg1/stanford-cars/master/labels_df.csv'

# !mkdir -p "./{output_data}"
# #images
# !wget {url_data} -P {output_data}
# !tar zxvf {train_file} {output_data}
# !tar zxvf {train_file} -C {output_data}
# #labels
# !wget {labels_url} -P {output_data}

# Configuration

In [ ]:
import pandas as pd
import json

In [ ]:
labels = pd.read_csv(output_data/'labels_df.csv')
labels.head()
labels.tail()

,filename,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class_id,class_name,is_test,filename_cropped,bbox_h,bbox_w
0,00001.jpg,39,116,569,375,14,Audi TTS Coupe 2012,0,cropped_00001.jpg,260,531
1,00002.jpg,36,116,868,587,3,Acura TL Sedan 2012,0,cropped_00002.jpg,472,833
2,00003.jpg,85,109,601,381,91,Dodge Dakota Club Cab 2007,0,cropped_00003.jpg,273,517
3,00004.jpg,621,393,1484,1096,134,Hyundai Sonata Hybrid Sedan 2012,0,cropped_00004.jpg,704,864
4,00005.jpg,14,36,133,99,106,Ford F-450 Super Duty Crew Cab 2012,0,cropped_00005.jpg,64,120


,filename,bbox_x1,bbox_y1,bbox_x2,bbox_y2,class_id,class_name,is_test,filename_cropped,bbox_h,bbox_w
16180,test_08037.jpg,49,57,1169,669,63,Chevrolet Sonic Sedan 2012,1,test_08037.jpg,613,1121
16181,test_08038.jpg,23,18,640,459,16,Audi V8 Sedan 1994,1,test_08038.jpg,442,618
16182,test_08039.jpg,33,27,602,252,17,Audi 100 Sedan 1994,1,test_08039.jpg,226,570
16183,test_08040.jpg,33,142,521,376,38,BMW Z4 Convertible 2012,1,test_08040.jpg,235,489
16184,test_08041.jpg,77,73,506,380,32,BMW X5 SUV 2007,1,test_08041.jpg,308,430


In [ ]:
from PIL import Image

def build_annotation_json_from_frame(frame):
  images = []
  annotations = []
  for _, row in frame.iterrows():
    im_id = row.filename.split('.')[0]
    
    im_size = Image.open(str(train_location / '00001.jpg')).size
    images.append({
        "file_name": row.filename,
        "id": im_id,
        'width':im_size[0],
        'height':im_size[1],
    })
    annotations.append({
        'image_id':im_id,
        'segmentation':[],
        'id':im_id,
        'bbox':[row.bbox_x1, row.bbox_y1, row.bbox_x2, row.bbox_y2],
        'category_id': row.class_id,
        'area': im_size[0]*im_size[1]
    })
  categories = list(frame[['class_id', 'class_name']]
                .groupby(['class_id', 'class_name'])
                .size()
                .reset_index()
                .drop(0, axis=1)
                .rename({'class_id': 'id', 'class_name': 'name'}, axis=1)
                .to_dict('index')
                .values())
  for c in categories:
    c['supercategory'] = 'car'
  return {
      'images': images,
      'annotations': annotations,
      'categories': categories,
      "info": {
            "description": "COCO 2017 Dataset",
            "url": "http://cocodataset.org",
            "version": "1.0",
            "year": 2017,
            "contributor": "COCO Consortium",
            "date_created": "2017/09/01"
        },
        "licenses": [
            {
                "url": "http://creativecommons.org/licenses/by-nc-sa/2.0/",
                "id": 1,
                "name": "Attribution-NonCommercial-ShareAlike License"
            },
            {
                "url": "http://creativecommons.org/licenses/by-nc/2.0/",
                "id": 2,
                "name": "Attribution-NonCommercial License"
            }
        ]
  }

In [ ]:
annotations_json_file = output_data / "annotations.json"
with open(annotations_json_file, "w") as p: 
     p.write(json.dumps(build_annotation_json_from_frame(labels[labels.is_test == 0])))

1595346

In [ ]:
img_norm_cfg = dict(
    mean=[102.9801, 115.9465, 122.7717], std=[1.0, 1.0, 1.0], to_rgb=False)


train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='Resize', img_scale=(1333, 800), keep_ratio=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='Normalize', **img_norm_cfg),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels']),
]
test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
        type='MultiScaleFlipAug',
        img_scale=(1333, 800),
        flip=False,
        transforms=[
            dict(type='Resize', keep_ratio=True),
            dict(type='RandomFlip'),
            dict(type='Normalize', **img_norm_cfg),
            dict(type='Pad', size_divisor=32),
            dict(type='ImageToTensor', keys=['img']),
            dict(type='Collect', keys=['img']),
        ])
]


cfg = Config.fromfile('mmdetection/configs/fcos/fcos_center_r50_caffe_fpn_gn_1x_4gpu.py.py')
project_main_dir = Path('./drive/My Drive/Colab Notebooks/Studia (DNN)/projekt/')

data_root = train_location
dataset_type = 'CocoDataset'
cfg.data_root = data_root

data = dict(
    imgs_per_gpu=1,
    workers_per_gpu=1,
    train=dict(
        type=dataset_type,
        ann_file=annotations_json_file,
        img_prefix=data_root,
        pipeline=train_pipeline),
    test=dict(
        type=dataset_type,
        ann_file=annotations_json_file,
        img_prefix=data_root,
        pipeline=test_pipeline),
    val=dict(
        type=dataset_type,
        ann_file=annotations_json_file,
        img_prefix=data_root,
        pipeline=test_pipeline))


distributed = False # niestety w Colabie nie będzie distributed
logger = get_root_logger(cfg.log_level)
logger.info('Distributed training: {}'.format(distributed))

model = build_detector(cfg.model, train_cfg=cfg.train_cfg, test_cfg=cfg.test_cfg)

cfg.work_dir = str(project_main_dir / 'work_dir' / datetime.now().strftime('%Y%m%d_%H%M%S'))
cfg['data'] = data
cfg.gpus = 1 
cfg['seed'] = 42

if cfg.get('cudnn_benchmark', False):
    torch.backends.cudnn.benchmark = True 

cfg.lr_config = dict(
    policy='step',
    warmup='linear',
    warmup_iters=500,
    warmup_ratio=1.0 / 3,
    step=[16, 22])

datasets = [build_dataset(cfg.data.train)]

2020-03-29 14:14:11,225 - mmdet - INFO - Distributed training: False
2020-03-29 14:14:11,542 - mmdet - INFO - load model from: open-mmlab://resnet50_caffe
2020-03-29 14:14:11,638 - mmdet - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: conv1.bias, layer1.0.conv1.bias, layer1.0.conv2.bias, layer1.0.conv3.bias, layer1.0.downsample.0.bias, layer1.1.conv1.bias, layer1.1.conv2.bias, layer1.1.conv3.bias, layer1.2.conv1.bias, layer1.2.conv2.bias, layer1.2.conv3.bias, layer2.0.conv1.bias, layer2.0.conv2.bias, layer2.0.conv3.bias, layer2.0.downsample.0.bias, layer2.1.conv1.bias, layer2.1.conv2.bias, layer2.1.conv3.bias, layer2.2.conv1.bias, layer2.2.conv2.bias, layer2.2.conv3.bias, layer2.3.conv1.bias, layer2.3.conv2.bias, layer2.3.conv3.bias, layer3.0.conv1.bias, layer3.0.conv2.bias, layer3.0.conv3.bias, layer3.0.downsample.0.bias, layer3.1.conv1.bias, layer3.1.conv2.bias, layer3.1.conv3.bias, layer3.2.conv1.bias, layer3.2.conv2.bias, layer

loading annotations into memory...
Done (t=0.03s)
creating index...
index created!


In [ ]:
def get_classes(frame):
  return list(frame[['class_id', 'class_name']]
                .groupby(['class_id', 'class_name'])
                .size()
                .reset_index()
                .drop(0, axis=1)
                .rename({'class_id': 'id'}, axis=1)
                .to_dict('index')
                .values())

model.CLASSES = get_classes(labels)

train_detector(
    model,
    datasets,
    cfg,
    distributed=False,
    validate=False)

2020-03-29 14:14:18,095 - mmdet - INFO - Start running, host: root@dd62a10bf7e0, work_dir: /content/drive/My Drive/Colab Notebooks/Studia (DNN)/projekt/work_dir/20200329_141411
2020-03-29 14:14:18,096 - mmdet - INFO - workflow: [('train', 1)], max: 12 epochs
2020-03-29 14:14:46,264 - mmdet - INFO - Epoch [1][50/8144]	lr: 0.00399, eta: 15:13:01, time: 0.561, data_time: 0.417, memory: 1947, loss_cls: 0.8317, loss_bbox: 4.5679, loss_centerness: 0.6782, loss: 6.0778
2020-03-29 14:15:13,119 - mmdet - INFO - Epoch [1][100/8144]	lr: 0.00465, eta: 14:53:25, time: 0.537, data_time: 0.404, memory: 1981, loss_cls: 0.4922, loss_bbox: 3.2159, loss_centerness: 0.6553, loss: 4.3634
2020-03-29 14:15:39,002 - mmdet - INFO - Epoch [1][150/8144]	lr: 0.00532, eta: 14:35:55, time: 0.518, data_time: 0.382, memory: 2008, loss_cls: 0.4793, loss_bbox: 3.4415, loss_centerness: 0.6631, loss: 4.5839
2020-03-29 14:16:04,195 - mmdet - INFO - Epoch [1][200/8144]	lr: 0.00599, eta: 14:21:14, time: 0.504, data_time: 0.

OSError: ignored

In [ ]:
from google.colab import output
output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

In [ ]:
torch.save(model.state_dict(), str(output_data/'model.mdl')